In [32]:
!rm -rf tmp/

In [17]:
!python3 generate_dataset.py --extract_info --extract_features

  0%|                                                  | 0/2212 [00:00<?, ?it/s]Processing 5085344787448740525
Video 5085344787448740525.tar.gz not exists - downloading
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 95.4M  100 95.4M    0     0  2984k      0  0:00:32  0:00:32 --:--:-- 3444k:00:12  0:00:29 2195k
Extracting video from 5085344787448740525.tar.gz
Processing tmp/5085344787448740525.tar.gz

100%|██████████████████████████████████| 73588/73588 [00:06<00:00, 12104.31it/s]
Saving info.json
  0%|                                       | 1/2212 [00:56<34:31:30, 56.21s/it]Processing 5085357672350628540
Video 5085357672350628540.tar.gz not exists - downloading
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 76  114M   76 87.8M    0     0   740k      0  0:02:39  

In [118]:
import gzip
import numpy as np

arr = np.load('features/5085344787448740525.npy')
f = gzip.GzipFile("features/5085344787448740525.npy.gz", "w")
np.save(file=f, arr=arr)
f.close()

In [119]:
f = gzip.GzipFile('features/5085344787448740525.npy.gz', "r")

arra = np.load(f)

In [106]:
import json 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np

class BOBSLDataset(Dataset):

    def __init__(self, 
                 use_pose=True,
                 use_hands=True,
                 use_face=False,
                 max_frame_diff=10,
                 sampling_rate=411,
                 max_frames=1000,
                 stride=1,
                 info_path='info.json',
                 data_path='data/bobsl', 
                 transform=None
                 ):
        
        self.use_pose = use_pose
        self.use_hands = use_hands
        self.use_face = use_face
        self.max_frame_diff = max_frame_diff
        self.max_frames = max_frames
        self.stride = stride
        self.sampling_rate = sampling_rate

        self.data_path = data_path
        self.info_path = info_path
        self.transform = transform
        self.data = self.load_data()

    def load_data(self):
        data = []
        with open(self.info_path, 'r') as file:
            metadata = json.load(file)

        for DOC in metadata:

            doc_id = DOC['document_ids']
            frame_ids = DOC['frame_ids']

            frame_ids = np.array(frame_ids).astype(int)

            frame_diff = np.diff(frame_ids)

            for i in range(0, len(frame_ids) - self.max_frames, self.stride):

                if frame_diff[i:i+self.max_frames].max() < self.max_frame_diff:

                    frames = frame_ids[i:i+self.max_frames]

                    start_time = int(i * self.sampling_rate)
                    end_time = int((i+self.max_frames) * self.sampling_rate)
            
                    data.append({
                        'doc_id': doc_id,
                        'frame_ids': frames,
                        'start_time': start_time,
                        'end_time': end_time
                    })

        return data
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        
        doc_id = self.data[idx]['doc_id']
        frames = self.data[idx]['frame_ids']
        start_time = self.data[idx]['start_time']
        end_time = self.data[idx]['end_time']
        
        array = np.load(f'features/{doc_id}.npy')

        return {
            'doc_id': doc_id,
            'frames': frames,
            'sign_segment': array[start_time:end_time],
            'start_time': start_time,
            'end_time': end_time,
        }


In [126]:
!accelerate launch ../pretraining/run_wav2vec2_pretraining_no_trainer.py \
	--dataset_name="librispeech_asr" \
	--dataset_config_names clean clean \
	--dataset_split_names validation \
	--model_name_or_path="patrickvonplaten/wav2vec2-base-v2" \
	--output_dir="./wav2vec2-pretrained-demo" \
	--max_train_steps="20000" \
	--num_warmup_steps="32000" \
	--gradient_accumulation_steps="8" \
	--learning_rate="0.005" \
	--weight_decay="0.01" \
	--max_duration_in_seconds="20.0" \
	--min_duration_in_seconds="2.0" \
	--logging_steps="1" \
	--saving_steps="10000" \
	--per_device_train_batch_size="8" \
	--per_device_eval_batch_size="8" \
	--adam_beta1="0.9" \
	--adam_beta2="0.98" \
	--adam_epsilon="1e-06" \
	--gradient_checkpointing \
	--mask_time_prob="0.65" \
	--mask_time_length="10"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
wandb: Currently logged in as: karahan-sahin (boun-pilab). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /Users/karahansahin/Documents/Research/sign2vec/sign2vec/wandb/run-20240613_142659-n8serte8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fast-mountain-7
wandb: ⭐️ View project at https://wandb.ai/boun-pilab/wav2vec2-pretrained-demo
wandb: 🚀 View run at https://wandb.ai/boun-pilab/wav2vec2-pretrained-dem

In [109]:
from torch.utils.data import DataLoader

train_dataset = BOBSLDataset(use_pose=True, use_hands=True, use_face=False, max_frames=1000, stride=1)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [112]:
len(train_dataset)

79445

In [110]:
train_sample = next(iter(train_loader))

In [111]:
train_sample['array'].shape

torch.Size([32, 411000])

In [93]:
import json
import numpy as np

stride = 2
max_frame_diff = 50
max_frames = 1000

data = []
with open('info.json', 'r') as file:
    metadata = json.load(file)

for DOC in metadata:
    print(DOC.keys())
    doc_id = DOC['document_ids']
    frame_ids = DOC['frame_ids']

    frame_ids = np.array(frame_ids).astype(int)

    frame_diff = np.diff(frame_ids)

    array = np.load(f'features/{doc_id}.npy')
    sample_rate = int(array.shape[0] / frame_diff.shape[0])

    for i in range(0, len(frame_ids) - max_frames, stride):

        if frame_diff[i:i+max_frames].max() < max_frame_diff:

            frames = frame_ids[i:i+max_frames]

            start_time = int(i * sample_rate)
            end_time = int((i+max_frames) * sample_rate)
    
            data.append({
                'doc_id': doc_id,
                'frame_ids': frames,
                'array': array[start_time:end_time]
            })

        else:
            
            # crop the sequence from max_frame_diff
            crops = np.where(frame_diff[i:i+max_frames] > max_frame_diff)[0]
            start_idx = crops[0]

            print('---', crops)
            # print('-',crops, start_idx, crops[1:])

            start_idx = 0
            for idx, crop in enumerate(crops.tolist()):
                if crop == 0:
                    continue

                start_time = int((start_idx+i) * sample_rate)
                end_time = int((start_idx+i+crop) * sample_rate)

                print('Crop - internal', (start_idx, crop), start_idx+i, start_idx+i+crop)
                print(array[start_time:end_time].shape)
                arr = np.pad(array[start_time:end_time], (((max_frames - crop) * sample_rate), (0)), 'constant', constant_values=0)
                print(arr.shape[0] == max_frames * sample_rate, arr.shape[0], max_frames * sample_rate)

                data.append({
                    'doc_id': doc_id,
                    'frame_ids': frame_ids[i+start_idx:i+crop+start_idx],
                    'array': arr
                })

                start_idx = crop    

                if idx == len(crops) - 1:
                    
                    start_time = int((i+start_idx) * sample_rate)
                    end_time = int((i+max_frames) * sample_rate)
                    
                    print('Crop - after', start_idx, max_frames, start_time, end_time)
                    print(array[start_time:end_time].shape)
                    arr_ = np.pad(array[start_time:end_time], (((max_frames - start_idx + 1) * sample_rate), (0)), 'constant', constant_values=0)
                        
                    print(arr_.shape[0] == max_frames * sample_rate, arr_.shape[0], max_frames * sample_rate)

                    data.append({
                        'doc_id': doc_id,
                        'frame_ids': frame_ids[i+start_idx:i+max_frames],
                        'array': arr_
                    })


                    
            
                print('=====')

dict_keys(['document_ids', 'face_missing', 'right_hand_missing', 'left_hand_missing', 'pose_missing', 'frame_ids'])
--- [  0 733]
Crop - internal (0, 733) 0 733
(301263,)
True 411000 411000
Crop - after 733 1000 301263 411000
(109737,)
False 219885 411000
=====
--- [731]
Crop - internal (0, 731) 2 733
(300441,)
True 411000 411000
Crop - after 731 1000 301263 411822
(110559,)
False 221529 411000
=====
--- [729]
Crop - internal (0, 729) 4 733
(299619,)
True 411000 411000
Crop - after 729 1000 301263 412644
(111381,)
False 223173 411000
=====
--- [727]
Crop - internal (0, 727) 6 733
(298797,)
True 411000 411000
Crop - after 727 1000 301263 413466
(112203,)
False 224817 411000
=====
--- [725]
Crop - internal (0, 725) 8 733
(297975,)
True 411000 411000
Crop - after 725 1000 301263 414288
(113025,)
False 226461 411000
=====
--- [723]
Crop - internal (0, 723) 10 733
(297153,)
True 411000 411000
Crop - after 723 1000 301263 415110
(113847,)
False 228105 411000
=====
--- [721]
Crop - internal (

In [58]:
len(data)

39726

In [54]:
411 / 3

137.0

In [127]:
!rm -rf tmp/